# Claims Triage - Training Pipeline
This notebook builds a basic ML classifier to predict claim urgency and risk.

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib

## Load and Label Data

In [ ]:
data = pd.read_csv('../data/claims_sample.csv')
data['urgency_label'] = data['claim_text'].apply(lambda x: 'High' if 'emergency' in x.lower() else 'Low')
data['risk_label'] = data['claim_text'].apply(lambda x: 'High' if 'surgery' in x.lower() or 'fracture' in x.lower() else 'Low')
data.head()

## Train Model for Urgency

In [ ]:
X = data['claim_text']
y_urgency = data['urgency_label']

X_train, X_test, y_train, y_test = train_test_split(X, y_urgency, test_size=0.2, random_state=42)

urgency_model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

urgency_model.fit(X_train, y_train)
y_pred = urgency_model.predict(X_test)
print("Urgency Classification Report:")
print(classification_report(y_test, y_pred))

joblib.dump(urgency_model, '../api/model/urgency_model.pkl')

## Train Model for Risk

In [ ]:
y_risk = data['risk_label']

X_train_risk, X_test_risk, y_train_risk, y_test_risk = train_test_split(X, y_risk, test_size=0.2, random_state=42)

risk_model = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression())
])

risk_model.fit(X_train_risk, y_train_risk)
y_pred_risk = risk_model.predict(X_test_risk)
print("Risk Classification Report:")
print(classification_report(y_test_risk, y_pred_risk))

joblib.dump(risk_model, '../api/model/risk_model.pkl')